Have to include some part about how I got the biggest playlist ever

In [18]:
# import pandas module 
import pandas as pd 
	
# making dataframe 
data = pd.read_csv('biggest_playlist_ever.csv', on_bad_lines='warn')

# output the dataframe 
data

,Spotify ID,Artist IDs,Track Name,Album Name,Artist Name(s),Release Date,Duration (ms),Popularity,Added By,Added At,...,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature
0,7dS5EaCoMnN7DzlpT6aRn2,2FXC3k01G6Gw61bmprjgqS,Take Me To Church,Hozier (Deluxe),Hozier,2014-05-20,241688,0,spotify:user:1130225252,2017-06-09T17:34:13Z,...,4,-5.303,0,0.0464,0.63400,0.000000,0.116,0.437,128.945,4
1,2V4bv1fNWfTcyRJKmej6Sj,"2KsP6tYLJlTBvSUxnwlVWa,1Bo8Afb2Qbjs4x6kJHyjle",Cooler Than Me - Single Mix,31 Minutes to Takeoff,"Mike Posner,Gigamesh",2010-08-09,213293,79,spotify:user:1130225252,2017-06-29T20:42:16Z,...,7,-4.630,0,0.0474,0.17900,0.000000,0.689,0.625,129.965,4
2,7KA4W4McWYRpgf0fWsJZWB,"4V8LLVI7PbaPR0K2TGSxFF,1U1el3k54VvEUzo3ybLPlM",See You Again (feat. Kali Uchis),Flower Boy,"Tyler, The Creator,Kali Uchis",2017-07-21,180386,89,spotify:user:1130225252,2018-07-19T15:34:54Z,...,6,-9.222,1,0.0959,0.37100,0.000007,0.109,0.620,78.558,4
3,3gbBpTdY8lnQwqxNCcf795,7EQ0qTo7fWT7DPxmxtSYEc,Pompeii,Bad Blood,Bastille,2013-01-01,214147,73,spotify:user:1130225252,2016-07-05T18:32:51Z,...,9,-6.383,1,0.0407,0.07550,0.000000,0.271,0.571,127.435,4
4,3ZFTkvIE7kyPt6Nu3PEa7V,"0EmeFodog0BfCgMzAIvKQp,7aBzpmFXB4WWpPl2F7RjBe",Hips Don't Lie (feat. Wyclef Jean),"Oral Fixation, Vol. 2 (Expanded Edition)","Shakira,Wyclef Jean",2005-11-28,218093,87,spotify:user:1130225252,2016-07-05T18:39:38Z,...,10,-5.892,0,0.0707,0.28400,0.000000,0.405,0.758,100.024,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5289,4Zcz6saEkOII3PlXd9gN3o,6CwfuxIqcltXDGjfZsMd9A,Hermit the Frog,The Family Jewels,MARINA,2009-07-15,215960,67,spotify:user:1130225252,2021-06-16T11:00:52Z,...,0,-4.545,1,0.0312,0.24300,0.000000,0.199,0.487,122.034,4
5290,61KpQadow081I2AsbeLcsb,1McMsnEElThX1knmY4oliG,deja vu,deja vu,Olivia Rodrigo,2021-04-01,215507,16,spotify:user:1130225252,2021-06-16T11:00:59Z,...,9,-7.236,1,0.1160,0.59300,0.000011,0.341,0.172,181.088,4
5291,5yorXJWdBan1Vlh116ZtQ7,6veh5zbFpm31XsPdjBgPER,WHOLE LOTTA MONEY,FOR CERTAIN,BIA,2020-12-11,156005,57,spotify:user:1130225252,2021-06-16T11:01:04Z,...,1,-5.019,1,0.3680,0.09040,0.000000,0.325,0.441,81.008,4
5292,11ZulcYY4lowvcQm4oe3VJ,"3PyJHH2wyfQK3WZrk9rpmP,6lay1nwbE6hTx1jivysUAL",Slumber Party (feat. Princess Nokia),DEMIDEVIL,"Ashnikko,Princess Nokia",2021-01-15,178404,73,spotify:user:1130225252,2021-06-16T11:01:10Z,...,11,-8.981,0,0.0795,0.00151,0.000039,0.101,0.563,105.012,4


In [19]:
#Removing columns that are not required
data.drop(['Artist IDs', 'Added By', 'Added At'], axis = 1, inplace = True)

data

,Spotify ID,Artist IDs,Track Name,Album Name,Artist Name(s),Release Date,Duration (ms),Popularity,Genres,Danceability,...,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature
0,7dS5EaCoMnN7DzlpT6aRn2,2FXC3k01G6Gw61bmprjgqS,Take Me To Church,Hozier (Deluxe),Hozier,2014-05-20,241688,0,"irish singer-songwriter,modern rock,pov: indie",0.566,...,4,-5.303,0,0.0464,0.63400,0.000000,0.116,0.437,128.945,4
1,2V4bv1fNWfTcyRJKmej6Sj,"2KsP6tYLJlTBvSUxnwlVWa,1Bo8Afb2Qbjs4x6kJHyjle",Cooler Than Me - Single Mix,31 Minutes to Takeoff,"Mike Posner,Gigamesh",2010-08-09,213293,79,"dance pop,pop,pop dance,pop rap,filter house,n...",0.768,...,7,-4.630,0,0.0474,0.17900,0.000000,0.689,0.625,129.965,4
2,7KA4W4McWYRpgf0fWsJZWB,"4V8LLVI7PbaPR0K2TGSxFF,1U1el3k54VvEUzo3ybLPlM",See You Again (feat. Kali Uchis),Flower Boy,"Tyler, The Creator,Kali Uchis",2017-07-21,180386,89,"hip hop,rap,colombian pop",0.558,...,6,-9.222,1,0.0959,0.37100,0.000007,0.109,0.620,78.558,4
3,3gbBpTdY8lnQwqxNCcf795,7EQ0qTo7fWT7DPxmxtSYEc,Pompeii,Bad Blood,Bastille,2013-01-01,214147,73,"metropopolis,modern rock,pop",0.679,...,9,-6.383,1,0.0407,0.07550,0.000000,0.271,0.571,127.435,4
4,3ZFTkvIE7kyPt6Nu3PEa7V,"0EmeFodog0BfCgMzAIvKQp,7aBzpmFXB4WWpPl2F7RjBe",Hips Don't Lie (feat. Wyclef Jean),"Oral Fixation, Vol. 2 (Expanded Edition)","Shakira,Wyclef Jean",2005-11-28,218093,87,"colombian pop,dance pop,latin pop,pop,rap kreyol",0.778,...,10,-5.892,0,0.0707,0.28400,0.000000,0.405,0.758,100.024,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5289,4Zcz6saEkOII3PlXd9gN3o,6CwfuxIqcltXDGjfZsMd9A,Hermit the Frog,The Family Jewels,MARINA,2009-07-15,215960,67,"metropopolis,pop,pov: indie,uk alternative pop",0.609,...,0,-4.545,1,0.0312,0.24300,0.000000,0.199,0.487,122.034,4
5290,61KpQadow081I2AsbeLcsb,1McMsnEElThX1knmY4oliG,deja vu,deja vu,Olivia Rodrigo,2021-04-01,215507,16,pop,0.439,...,9,-7.236,1,0.1160,0.59300,0.000011,0.341,0.172,181.088,4
5291,5yorXJWdBan1Vlh116ZtQ7,6veh5zbFpm31XsPdjBgPER,WHOLE LOTTA MONEY,FOR CERTAIN,BIA,2020-12-11,156005,57,"rap latina,trap queen",0.897,...,1,-5.019,1,0.3680,0.09040,0.000000,0.325,0.441,81.008,4
5292,11ZulcYY4lowvcQm4oe3VJ,"3PyJHH2wyfQK3WZrk9rpmP,6lay1nwbE6hTx1jivysUAL",Slumber Party (feat. Princess Nokia),DEMIDEVIL,"Ashnikko,Princess Nokia",2021-01-15,178404,73,"alt z,escape room,alternative r&b,lgbtq+ hip h...",0.964,...,11,-8.981,0,0.0795,0.00151,0.000039,0.101,0.563,105.012,4


In [22]:
# #Rearranging the columns to make more sense (lol change this comment)
data = data[['Track Name', 'Album Name', 'Artist Name(s)', 'Release Date', 'Popularity', 'Duration (ms)', 'Acousticness', 'Danceability', 'Energy', 'Instrumentalness', 'Key', 'Liveness', 'Loudness', 'Mode', 'Speechiness', 'Tempo', 'Time Signature', 'Valence', 'Genres']]

# Spotify ID,Artist IDs,Track Name,Album Name,Artist Name(s),Release Date,Duration (ms),Popularity,Added By,Added At,Genres,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature

data

,Track Name,Album Name,Artist Name(s),Release Date,Popularity,Duration (ms),Acousticness,Danceability,Energy,Instrumentalness,Key,Liveness,Loudness,Mode,Speechiness,Tempo,Time Signature,Valence,Genres
0,Take Me To Church,Hozier (Deluxe),Hozier,2014-05-20,0,241688,0.63400,0.566,0.664,0.000000,4,0.116,-5.303,0,0.0464,128.945,4,0.437,"irish singer-songwriter,modern rock,pov: indie"
1,Cooler Than Me - Single Mix,31 Minutes to Takeoff,"Mike Posner,Gigamesh",2010-08-09,79,213293,0.17900,0.768,0.820,0.000000,7,0.689,-4.630,0,0.0474,129.965,4,0.625,"dance pop,pop,pop dance,pop rap,filter house,n..."
2,See You Again (feat. Kali Uchis),Flower Boy,"Tyler, The Creator,Kali Uchis",2017-07-21,89,180386,0.37100,0.558,0.559,0.000007,6,0.109,-9.222,1,0.0959,78.558,4,0.620,"hip hop,rap,colombian pop"
3,Pompeii,Bad Blood,Bastille,2013-01-01,73,214147,0.07550,0.679,0.715,0.000000,9,0.271,-6.383,1,0.0407,127.435,4,0.571,"metropopolis,modern rock,pop"
4,Hips Don't Lie (feat. Wyclef Jean),"Oral Fixation, Vol. 2 (Expanded Edition)","Shakira,Wyclef Jean",2005-11-28,87,218093,0.28400,0.778,0.824,0.000000,10,0.405,-5.892,0,0.0707,100.024,4,0.758,"colombian pop,dance pop,latin pop,pop,rap kreyol"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5289,Hermit the Frog,The Family Jewels,MARINA,2009-07-15,67,215960,0.24300,0.609,0.679,0.000000,0,0.199,-4.545,1,0.0312,122.034,4,0.487,"metropopolis,pop,pov: indie,uk alternative pop"
5290,deja vu,deja vu,Olivia Rodrigo,2021-04-01,16,215507,0.59300,0.439,0.610,0.000011,9,0.341,-7.236,1,0.1160,181.088,4,0.172,pop
5291,WHOLE LOTTA MONEY,FOR CERTAIN,BIA,2020-12-11,57,156005,0.09040,0.897,0.371,0.000000,1,0.325,-5.019,1,0.3680,81.008,4,0.441,"rap latina,trap queen"
5292,Slumber Party (feat. Princess Nokia),DEMIDEVIL,"Ashnikko,Princess Nokia",2021-01-15,73,178404,0.00151,0.964,0.398,0.000039,11,0.101,-8.981,0,0.0795,105.012,4,0.563,"alt z,escape room,alternative r&b,lgbtq+ hip h..."
